In [48]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [170]:
url = 'https://loyolagreyhounds.com/sports/womens-basketball/stats/2022-23'
path='https://loyolagreyhounds.com'

data_list = []
columns_overall = []
dict_colspan = {}
index = 0

soup = BeautifulSoup(html_content,'html')
main = soup.find('main',class_='main-content-placeholder')
table_seasons = main.find('select',id='ctl00_cplhMainContent_seasons_ddl')
data_years = table_seasons.find_all('option',value=True)
years = [path+data['value'] for data in data_years[1:]]

for year in years:
    response = requests.get(year)
    soup = BeautifulSoup(response.content,'html')
    print(f'Successfully received {url}')
    
    main = soup.find('main',class_='main-content-placeholder')
    overall = main.find('section',id='individual-overall')
    table = overall.find('table','sidearm-table')
    table_head = table.find('thead')
    headers_label = table_head.find_all('tr')[0]
    headers_column = table_head.find_all('tr')[1]

    body = table.find('tbody')
    players = body.find_all('tr')

    for player in players[:-1]:
        infos = player.find_all('td')
        data = [info.text.strip() for info in infos[:-1]]
        try:
            bio = infos[-1].find('a')['href']
            link= path+bio
        except TypeError:
            link=None
        data.append(link)
        data_list.append(data)

for header in headers_label.find_all("th"):
    colspan_value = int(header.get("colspan", 1)) #if header dont have a colspan, colspan_value get 1 - 
    header_text = header.get_text()
    dict_colspan[header_text] = colspan_value
    
data_headers = headers_column.find_all('th')
for span,text in zip(dict_colspan.values(),dict_colspan.keys()):
    data = []
    if(span <= 1):
        columns_overall.append(text)
    else:
        data= [head.text.strip() + "_" + text for head in data_headers[index:index+span]]
        index += span
    columns_overall.extend(data)  # Yeni verileri ana liste içine ekler 
       
df_overall = pd.DataFrame(data_list,columns = columns_overall)

Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2022-23
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2021-22
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2020-21
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2019-20
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2018-19
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2017-18
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2016-17
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2015-16
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2014-15
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2013-14
Successfully received https://loyolagreyhounds.com/sports/womens-basketball/stats/2012-13


In [263]:
def swap_ids(link1, link2):
    # Extract player IDs from the links
    id1_start = link1.rfind("=") + 1
    id1 = link1[id1_start:]
    
    id2_start = link2.rfind("/") + 1
    id2 = link2[id2_start:]
    
    # Create new links by swapping the IDs
    new_link = link2.replace(id2, id1)
    
    return new_link

In [335]:
def get_links(df_overall):
    links = []
    for bio_link in df_overall['Bio Link']:
        
        try:
            r = requests.head(bio_link, allow_redirects=True)
            response = requests.get(bio_link)
            
            if(response.status_code==404 and r.url!=bio_link):
                bio_link = swap_ids(r.url, bio_link)
                print("new link - ",bio_link,"appended.")
            elif(response.status_code==404 and r.url==bio_link): #some players' pages are sending 404, and their page on redirected page
                bio_link = None
                print("Pages are sending 404\t None values appended.")
            links.append(bio_link)
                
        except requests.exceptions.MissingSchema:
            print('MissingSchema error\t None values appended.')
            links.append(None)
    return links

In [336]:
links = get_links(df_overall)
print("\nPlayers' pages have been edited and data is ready to be pulled.")

new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/lex-therien/12242 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/koi-sims/12239 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/kelly-ratigan/12237 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/caramina-ta-ntilde-edo/12240 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/meliah-van-otoo/12243 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/laura-salmer-oacute-n/12238 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/cristina-garcia/12234 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/marta-espinal/12233 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basketball/roster/ava-therien/12241 appended.
new link -  https://loyolagreyhounds.com/sports/womens-basket

In [339]:
detail_header = None
detail_players=[]

for ind,link in enumerate(links):
    
    if link is None:
        print("None value encountered. Skipping...")
        detail_player = [None for i in range(len(details))]
        detail_players.append(detail_player)
        continue
    
    response = requests.get(link)
    soup = BeautifulSoup(response.content,'html')
    fields = soup.find('div',class_='sidearm-roster-player-fields')
    details = fields.find_all('li',class_='large-6')
    detail_player = [detail.find('dd').text for detail in details]
    print(detail_player)
    detail_players.append(detail_player)
      
    if detail_header is None and len(detail_player)>5:
        columns_details = [detail.find('dt').text for detail in details]

    print(link," is done.\n")


['Forward', '6-1', 'Junior', 'Cherokee', 'Marlton, N.J.']
https://loyolagreyhounds.com/sports/womens-basketball/roster/lex-therien/12242  is done.

['Guard', '5-10', 'Graduate Student', 'McDonogh School', 'Baltimore, Md.']
https://loyolagreyhounds.com/sports/womens-basketball/roster/taleah-dixon/11617  is done.

['Guard', '5-8', 'Graduate Student', 'Herndon', 'Herndon, Va.', "St. Joseph's"]
https://loyolagreyhounds.com/sports/womens-basketball/roster/devyne-newman/11619  is done.

['Forward', '6-0', 'Junior', 'Archbishop Spalding', 'Glen Burnie, Md.', 'Albany']
https://loyolagreyhounds.com/sports/womens-basketball/roster/koi-sims/12239  is done.

['Guard', '5-7', 'Senior', 'Highland', 'Highland, N.Y.']
https://loyolagreyhounds.com/sports/womens-basketball/roster/bri-rozzi/11620  is done.

['Guard', '5-8', 'Sophomore', 'Saint Joseph', 'South Bend, Ind.']
https://loyolagreyhounds.com/sports/womens-basketball/roster/kelly-ratigan/12237  is done.

['Guard', '5-10', 'Senior', "St. John's Co

In [405]:
df_overall_final = pd.concat([df_overall,pd.DataFrame(detail_players,columns=columns_details)],axis=1)
df_overall_final

,#,Player,GP,GS,TOT_Minutes,AVG_Minutes,FGM_FG,FGA_FG,FG%_FG,3PT_3PT,...,TO,STL,BLK,Bio Link,Position,Height,Class,Highschool,Hometown,Prev School
0,45,"Therien, Lex\n45 Therien, Lex",31,31,965,31.1,187,381,.491,2,...,103,49,7,https://loyolagreyhounds.com/sports/womens-bas...,Forward,6-1,Junior,Cherokee,"Marlton, N.J.",None
1,04,"Dixon, Taleah\n04 Dixon, Taleah",30,30,969,32.3,138,395,.349,44,...,76,44,1,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-10,Graduate Student,McDonogh School,"Baltimore, Md.",None
2,05,"Newman, Devyne\n05 Newman, Devyne",20,13,497,24.9,67,170,.394,8,...,26,6,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-8,Graduate Student,Herndon,"Herndon, Va.",St. Joseph's
3,10,"Sims, Koi\n10 Sims, Koi",20,19,536,26.8,58,132,.439,0,...,42,17,18,https://loyolagreyhounds.com/sports/womens-bas...,Forward,6-0,Junior,Archbishop Spalding,"Glen Burnie, Md.",Albany
4,22,"Rozzi, Bri\n22 Rozzi, Bri",27,23,746,27.6,55,155,.355,20,...,49,16,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-7,Senior,Highland,"Highland, N.Y.",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,22,"Butler, Brittany\n22 Butler, Brittany",3,0,7,2.3,1,3,.333,0,...,0,0,1,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-9,Sophomore,Howard,"Elkridge, Md.",None
149,01,"Robinson, Bryanna\r\n ...",6,0,25,4.2,0,10,.000,0,...,2,1,0,None,None,None,None,None,None,None
150,42,"Perry, Jeneh\n42 Perry, Jeneh",5,0,17,3.4,0,3,.000,0,...,3,2,0,https://loyolagreyhounds.com/sports/womens-bas...,Forward,Senior,Girls Preparatory School,"Chattanooga, Tenn.",None,None
151,35,"Brown, Nai\n35 Brown, Nai",6,1,19,3.2,0,2,.000,0,...,5,1,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-9,Junior,Sanford School,"Newark, Del.",None


In [406]:
df_overall_final['Player'] = df_overall_final['Player'].apply(lambda x: x.split("\n")[0].strip())
df_overall_final

,#,Player,GP,GS,TOT_Minutes,AVG_Minutes,FGM_FG,FGA_FG,FG%_FG,3PT_3PT,...,TO,STL,BLK,Bio Link,Position,Height,Class,Highschool,Hometown,Prev School
0,45,"Therien, Lex",31,31,965,31.1,187,381,.491,2,...,103,49,7,https://loyolagreyhounds.com/sports/womens-bas...,Forward,6-1,Junior,Cherokee,"Marlton, N.J.",None
1,04,"Dixon, Taleah",30,30,969,32.3,138,395,.349,44,...,76,44,1,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-10,Graduate Student,McDonogh School,"Baltimore, Md.",None
2,05,"Newman, Devyne",20,13,497,24.9,67,170,.394,8,...,26,6,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-8,Graduate Student,Herndon,"Herndon, Va.",St. Joseph's
3,10,"Sims, Koi",20,19,536,26.8,58,132,.439,0,...,42,17,18,https://loyolagreyhounds.com/sports/womens-bas...,Forward,6-0,Junior,Archbishop Spalding,"Glen Burnie, Md.",Albany
4,22,"Rozzi, Bri",27,23,746,27.6,55,155,.355,20,...,49,16,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-7,Senior,Highland,"Highland, N.Y.",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,22,"Butler, Brittany",3,0,7,2.3,1,3,.333,0,...,0,0,1,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-9,Sophomore,Howard,"Elkridge, Md.",None
149,01,"Robinson, Bryanna",6,0,25,4.2,0,10,.000,0,...,2,1,0,None,None,None,None,None,None,None
150,42,"Perry, Jeneh",5,0,17,3.4,0,3,.000,0,...,3,2,0,https://loyolagreyhounds.com/sports/womens-bas...,Forward,Senior,Girls Preparatory School,"Chattanooga, Tenn.",None,None
151,35,"Brown, Nai",6,1,19,3.2,0,2,.000,0,...,5,1,0,https://loyolagreyhounds.com/sports/womens-bas...,Guard,5-9,Junior,Sanford School,"Newark, Del.",None


In [407]:
df_overall_final.to_csv('df_overall_final.csv',index=False)